<p style="align: center;">
    <img align=center src="../img/dls_logo.jpg" width=500 height=500>
</p>

<h1 style="text-align: center;">
    Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ
</h1>

---

<h1 style="text-align: center;">
    <b>Градиентный спуск и линейные модели</b>
</h1>

В этом ноутбуке мы попробуем реализовать свой градиентный спуск на основе модели линейной регрессии и сравним свою реализацию с библиотечной.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

## Построение модели

Модель нашей линейной регрессии:

In [ ]:
# в этих переменных будут лежать веса, которые мы оценим
# w - веса модели, на которые умножаются признаки
w = None
# b - bias, который добавляется к итоговому результату
b = None


def mse(preds, y):
    """
    Возвращает среднеквадратичную ошибку между preds и y.
    """
    
    return ((preds - y)**2).mean()


def solve_weights(X, y):
    """
    Находит параметры w, b по методу наименьших квадратов для X и y.
    Решает систему линейных уравнений, к которым приводит метод наименьших 
    квадратов, для признаков X и значений y.
    """
    
    # ключевое слово global позволяет нам использовать
    # глобальные переменные, определенные в начале ячейки
    global w, b
    
    n = X.shape[0]
    # добавляем к признакам фиктивную размерность, чтобы было удобнее находить bias
    bias = np.ones((n, 1))
    X_b = np.append(bias, X, axis=1)
    
    # используем формулу из метода наименьших квадратов
    # w_full сожержит коэффициенты w и b, так как мы добавили фиктивную размерность к признакам
    w_full = np.linalg.inv(X_b.T @ X_b) @ X_b.T @ y
    
    # мы разделяем bias, который лежал в начале вектора w_full, и веса модели w
    w = w_full[1:]
    b = np.array([w_full[0]])
    # нам не нужно возвращать w и b, так как они уже лежат в глобальных переменных
    

def predict(X):
    """
    Предсказывает значения y, используя текущие параметры модели w и b
    """
    
    return X @ w + b


def grad_descent(X, y, lr, num_iters=100):
    """
    Находит приближённые значения параметров модели, используя градиентный спуск.
    Функция потерь (ошибки) для данной реализации спуска - MSE.
    Возвращаемое значение - список значений функции потерь на каждом шаге.
    """
    
    # ключевое слово global позволяет нам использовать
    # глобальные переменные, определенные в начале ячейки
    global w, b
    w = np.random.rand(X.shape[1])
    b = np.array(np.random.rand(1))
    
    losses = []
    
    n = X.shape[0]
    
    for cur_iter in range(num_iters):
        preds = predict(X)
        losses.append(mse(preds, y))
        
        w_grad = np.zeros_like(w)
        b_grad = 0
        
        for sample, prediction, label in zip(X, preds, y):
            w_grad += 2 * (prediction - label) * sample
            b_grad += 2 * (prediction - label)
            
        w -= lr * w_grad
        b -= lr * b_grad

    return losses

Подробнее рассмотрим формулы, которые используются в градиентном спуске. Наша функция потерь:

$$
L(\hat{y}) = \sum_{i = 1}^{n}( \hat{y}_{i} - y_{i} )^{2}
$$

Найдём производную:

$$
\frac{dL(\hat{y})}{d\hat{y}} = \sum_{i = 1}^{n}2(\hat{y}_{i} - y_{i})
$$

где $\hat{y}$ - это вектор предсказаний, а $y$ - вектор значений.

Если у нас есть только два признака, то по определению нашей модели:

$$
\hat{y}_{i} = w_1 \cdot x_{i1} + w_2 \cdot x_{i2} + b
$$

Подставим в формулу для функции потерь и возьмём производную:

$$
\frac{\partial L(\hat{y})}{ \partial w_1} = \sum_{i = 1}^{n} \frac{\partial (( \hat{y}_{i} - y_{i} )^{2})}{\partial \hat{y_i}} \cdot \frac{\partial \hat{y_i}}{\partial w_1}  =  
\sum_{i = 1}^{n} 2 (\hat{y_i} - y) \cdot x_{i1}
$$

В формуле есть суммирование по всем строчкам $X$ ($x_i$ это $i$-ая строчка $X$, в которой хранятся признаки для $i$-го наблюдения), в коде ему соответствует внешний цикл, итерирующийся по всем наблюдениям. Внутренний цикл нужен для получения производных по всем весам $w_i$, которых в общем случае может быть произвольное количество.

В итоге выполнения кода:

$$
w\_grad = (\frac{\partial L(\hat{y})}{\partial w_1} , \frac{\partial L(\hat{y})}{\partial w_2}, \frac{\partial L(\hat{y})}{\partial w_3}, \ldots) = \nabla L
$$ 

Для обновления весов мы вычитаем градиент, передвигаясь в направлении скорейшего убывания функции:

$$
w = w - lr \cdot \nabla L
$$

## Получение данных

In [ ]:
def generate_data(range_, a, b, std, num_points=100):
    """Генерирует данные в заданном промежутке, которые подчиняются
    зависимости y = a * x + b + е, где е - нормально распределена со
    стандартным отклонением std и нулевым средним.
    """
    
    X_train = np.random.random(num_points) * (range_[1] - range_[0]) + range_[0]
    y_train = a * X_train + b + np.random.normal(0, std, size=X_train.shape)
    
    return X_train, y_train

In [ ]:
# зададим параметры для искусственных данных
real_a = 0.34
real_b = 13.7
real_std = 7

# генерируем данные в промежутке от 0 до 150 с параметрами, которые мы задали выше
X_train, y_train = generate_data([0, 150], real_a, real_b, real_std)

# просто выведем табличку с данными
pd.DataFrame({'X': X_train, 'Y': y_train}).head()

In [ ]:
plt.scatter(X_train, y_train, c='black')
plt.plot(X_train, real_a * X_train + real_b)
plt.show()

## Решение с помощью линейной алгебры

In [ ]:
# используем функцию, написанную выше, чтобы найти w и b
# с помощтю метода наименьших квадратов
solve_weights(X_train.reshape(-1, 1), y_train)

In [ ]:
# полученные веса лежат в глобальных переменных, выведем их
w, b

Полученные веса очень похожи на те, которые мы задавали при генерации данных. Значит модель получилась хорошей.

In [ ]:
# выведем данные, истинную зависимость и зависимость,
# полученную нами с помощью метода наименьших квадратов
plt.scatter(X_train, y_train, c='red')
plt.plot(X_train, real_a * X_train + real_b)
plt.plot(X_train, X_train.reshape(-1, 1) @ w + b)
plt.show()

## Решение с помощью градиентного спуска

In [ ]:
# найдём параметры с помощью градиентного спуска
# чтобы проследить за обучением, мы записываем значение
# функции ошибки на каждом шаге и после выводим
losses = grad_descent(X_train.reshape(-1, 1), y_train, 1e-9, 15000)

In [ ]:
# полученные веса лежат в глобальных переменных, выведем их
w, b

Веса модели получились не похожи, на то, что мы задавали при генерации данных. Модель намного хуже.

Стоит отметить, что хуже всего был подобран свободный член $b$, это связано с тем, что данные не нормализованы и параметры $a$ и $b$ имеют очень разные модули, а шаги, которые делает градиентный спуск для обоих параметров одного порядка. Это приводит к тому, что меньший по модулю параметр $a$ быстро подбирается, а параметр почти $b$ перестает изменяться.

In [ ]:
# выведем график функции потерь 
plt.plot(losses)
plt.show()

losses[-1]

In [ ]:
# выведем данные, истинную зависимость и зависимость, полученную нами
plt.scatter(X_train, y_train, c='red')
plt.plot(X_train, real_a * X_train + real_b)
plt.plot(X_train, X_train.reshape(-1, 1) @ w + b)
plt.show()

Градиентный спуск восстановил зависимость хуже, чем метод наименьших квадратов, это вызвано тем, что:

* данные **не нормализованы** (подробнее о нормализации в домашнем ноутбуке)

* в **методе наименьших квадратов** мы получали решение **аналитически**, поэтому оно гарантировано является наилучшим, в то время как градиентный спуск находит решение лишь приближённо

Возникает вопрос, зачем использовать **градиентный спуск**, если он хуже **аналитических** мтеодов? Дело в том, что оптимизация большого количества весов в **нейронных сетях** сликшом сложная задача, которая не может быть решена **аналитически**.

## Данные посложнее

Загрузим с помощью `pandas` реальные данные и попробуем найти параметры зависимости с помощью метода наименьших квадратов и градиентного спуска, как и в предыдущем примере (так как наш код универсален, нам просто нужно просто вызвать те же функции).

In [ ]:
df = pd.read_csv("data/data.csv")

In [ ]:
# так как данные многомерные, мы не можем построить график, как в предыдущем примере, 
# чтобы увидеть зависимость глазами, поэтому мы просто выведем первые строки таблицы
df.head()

In [ ]:
# разделим данные на признаки и значения
data, label = np.array(df)[:, 1:5], np.array(df)[:, 5]

### Решение с помощью линейной алгебры

In [ ]:
# используем функцию, написанную выше, чтобы найти
# w и b с помощью метода наименьших квадратов
solve_weights(data, label)

In [ ]:
# полученные веса лежат в глобальных переменных, выведем их
w, b

In [ ]:
# выведем значение функции ошибки, чтобы позже сравнить с результатом градиентного спуска
mse(predict(data), label)

### Решение с помощью градиентного спуска

In [ ]:
# найдём параметры с помощью градиентного спуска
# чтобы проследить за обучением, мы записываем значение
# функции ошибки на каждом шаге и после выводим
losses = grad_descent(data, label, 1e-9, 500)

In [ ]:
# полученные веса лежат в глобальных переменных, выведем их
w, b

In [ ]:
# выведем график функции потерь 
plt.plot(losses)
plt.show()

In [ ]:
# выведем значение функции ошибки
mse(predict(data), label)

Как мы видим, **градиентный спуск** опять нашел значительно более плохое решение. Если нормализовать данные, то **градиентный спуск** будет сходиться лучше и разница будет не такой заметной. 

В домашнем задании вы научитесь нормализовывать данные. После этого вы можете вернуться в этот ноутбук и запустить градиентный спуск, предварительно использовав нормализацию.